In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from textblob import TextBlob

In [4]:
artist_details = pd.read_csv('../viz_data/artist_details.csv',index_col=0)
track_details= pd.read_csv('../data//truncated_tracks.csv',index_col=0)
track_feature= pd.read_csv('../viz_data//tracks_feature_df.csv',index_col=0)

In [5]:
artist_details.head()

artist_names                track_id  \
0       Bob Dylan  3AhXZa8sUQht0UEdBJgpGc   
1         Nirvana  5ghIJDpPoe3CfHMGu71E6T   
2     The Beatles  0hKRSZhUGEhKU6aNSPBACZ   
3  The Beach Boys  5Qt4Cc66g24QWwGP3YYV9y   
4     Chuck Berry  2QfiRTz5Yc8DdShCxG1tB2   

                                               genre  popularity  
0  album rock, classic rock, country rock, folk, ...        71.0  
1     alternative rock, grunge, permanent wave, rock        81.0  
2  beatlesque, british invasion, classic rock, me...        83.0  
3  baroque pop, brill building pop, classic rock,...        72.0  
4      classic rock, rock, rock-and-roll, rockabilly        65.0

In [6]:
track_details.head()

track_name  track_popularity album_released_date  \
0                 Like a Rolling Stone                69          1965-08-30   
1              Smells Like Teen Spirit                78          1991-09-26   
2  A Day In The Life - Remastered 2009                62          1967-06-01   
3               Good Vibrations (Mono)                20          1967-09-18   
4                      Johnny B. Goode                75          1959-07-01   

                       id                album_id  \
0  3AhXZa8sUQht0UEdBJgpGc  6YabPKtZAjxwyWbuO9p4ZD   
1  5ghIJDpPoe3CfHMGu71E6T  2guirTSEqLizK7j9i1MTTZ   
2  0hKRSZhUGEhKU6aNSPBACZ  6QaVfG1pHYl1z15ZxkvVDW   
3  5Qt4Cc66g24QWwGP3YYV9y  3yGRRfowVUIj29zEwgLYY7   
4  2QfiRTz5Yc8DdShCxG1tB2  6eedtCtCjibu80yOhylSGL   

                                          album_name album_type  \
0                               Highway 61 Revisited      album   
1                             Nevermind (Remastered)      album   
2  Sgt. Pepper's Lonely Hearts Club Band (Remaste...      album   
3                       Smiley Smile (Mono & Stereo)      album   
4                                    Berry Is On Top      album   

                    artist_id  
0  ['74ASZWbe4lXaubB36ztrGX']  
1  ['6olE6TJLqED3rqDCT0FyPh']  
2  ['3WrFJ7ztbogyGnTHbHJFl2']  
3  ['3oDbviiivRWhXwIE8hxkVV']  
4  ['293zczrfYafIItmnmM3coR']

In [7]:
track_feature.head()

danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.482   0.721    0    -6.839     1       0.0321      0.731000   
1         0.502   0.912    1    -4.556     1       0.0564      0.000025   
2         0.364   0.457    4   -14.162     0       0.0675      0.290000   
3         0.398   0.413    1   -10.934     1       0.0388      0.082200   
4         0.534   0.803   10    -9.129     1       0.0743      0.741000   

   instrumentalness  liveness  valence    tempo            type  \
0          0.000000    0.1890    0.557   95.263  audio_features   
1          0.000173    0.1060    0.720  116.761  audio_features   
2          0.000106    0.9220    0.175  163.219  audio_features   
3          0.000025    0.0891    0.331  133.574  audio_features   
4          0.000061    0.3070    0.969  167.983  audio_features   

                       id                                   uri  \
0  3AhXZa8sUQht0UEdBJgpGc  spotify:track:3AhXZa8sUQht0UEdBJgpGc   
1  5ghIJDpPoe3CfHMGu71E6T  spotify:track:5ghIJDpPoe3CfHMGu71E6T   
2  0hKRSZhUGEhKU6aNSPBACZ  spotify:track:0hKRSZhUGEhKU6aNSPBACZ   
3  5Qt4Cc66g24QWwGP3YYV9y  spotify:track:5Qt4Cc66g24QWwGP3YYV9y   
4  2QfiRTz5Yc8DdShCxG1tB2  spotify:track:2QfiRTz5Yc8DdShCxG1tB2   

                                          track_href  \
0  https://api.spotify.com/v1/tracks/3AhXZa8sUQht...   
1  https://api.spotify.com/v1/tracks/5ghIJDpPoe3C...   
2  https://api.spotify.com/v1/tracks/0hKRSZhUGEhK...   
3  https://api.spotify.com/v1/tracks/5Qt4Cc66g24Q...   
4  https://api.spotify.com/v1/tracks/2QfiRTz5Yc8D...   

                                        analysis_url  duration_ms  \
0  https://api.spotify.com/v1/audio-analysis/3AhX...       369600   
1  https://api.spotify.com/v1/audio-analysis/5ghI...       301920   
2  https://api.spotify.com/v1/audio-analysis/0hKR...       337413   
3  https://api.spotify.com/v1/audio-analysis/5Qt4...       219147   
4  https://api.spotify.com/v1/audio-analysis/2Qfi...       161560   

   time_signature  speechiness_binned  danceability_binned  \
0               4                   0                    1   
1               4                   0                    1   
2               4                   0                    1   
3               4                   0                    1   
4               4                   0                    1   

   instrumentalness_binned  
0                        0  
1                        0  
2                        0  
3                        0  
4                        0

In [8]:

track_feature =track_feature.drop(columns =['uri','track_href','analysis_url'])

In [9]:
artist_details.rename(columns={'track_id':'id'},inplace=True)

In [10]:
merge_data= artist_details.merge(track_feature,how='left',on='id')

In [11]:
final_data=merge_data.merge(track_details,how='left',on='id')

In [13]:
final_data.shape

(9732, 28)

In [15]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9732 entries, 0 to 9731
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   artist_names             9658 non-null   object 
 1   id                       9732 non-null   object 
 2   genre                    9317 non-null   object 
 3   popularity               9732 non-null   float64
 4   danceability             9732 non-null   float64
 5   energy                   9732 non-null   float64
 6   key                      9732 non-null   int64  
 7   loudness                 9732 non-null   float64
 8   mode                     9732 non-null   int64  
 9   speechiness              9732 non-null   float64
 10  acousticness             9732 non-null   float64
 11  instrumentalness         9732 non-null   float64
 12  liveness                 9732 non-null   float64
 13  valence                  9732 non-null   float64
 14  tempo                   

In [16]:
final_data['genre']=final_data['genre'].fillna("")

In [17]:
def genre_preprocess(df):
    '''
    Preprocess the genre data
    '''
    df['genres_list'] = df['genre'].apply(lambda x: x.split(" "))
    return df
final_data = genre_preprocess(final_data)
final_data['genres_list'].head()

0    [album, rock,, classic, rock,, country, rock,,...
1    [alternative, rock,, grunge,, permanent, wave,...
2    [beatlesque,, british, invasion,, classic, roc...
3    [baroque, pop,, brill, building, pop,, classic...
4    [classic, rock,, rock,, rock-and-roll,, rockab...
Name: genres_list, dtype: object

In [18]:
final_data.head(10)

artist_names                      id  \
0           Bob Dylan  3AhXZa8sUQht0UEdBJgpGc   
1             Nirvana  5ghIJDpPoe3CfHMGu71E6T   
2         The Beatles  0hKRSZhUGEhKU6aNSPBACZ   
3      The Beach Boys  5Qt4Cc66g24QWwGP3YYV9y   
4         Chuck Berry  2QfiRTz5Yc8DdShCxG1tB2   
5     Aretha Franklin  7s25THrKz86DM225dOYwnr   
6        The Ronettes  2G2YzndIA6jeWFPBXhUjh5   
7         Marvin Gaye  3Um9toULmYFGCpvaIPFw7l   
8         Marvin Gaye  1tqT6DhmsrtQgyCKUwotiw   
9  The Rolling Stones  2PzU4IB8Dr6mxV3lHuaG34   

                                               genre  popularity  \
0  album rock, classic rock, country rock, folk, ...        71.0   
1     alternative rock, grunge, permanent wave, rock        81.0   
2  beatlesque, british invasion, classic rock, me...        83.0   
3  baroque pop, brill building pop, classic rock,...        72.0   
4      classic rock, rock, rock-and-roll, rockabilly        65.0   
5  classic soul, jazz blues, memphis soul, soul, ...        70.0   
6  brill building pop, classic girl group, rock-a...        58.0   
7  classic soul, funk, indie r&b, motown, norther...        72.0   
8  classic soul, funk, indie r&b, motown, norther...        72.0   
9               british invasion, classic rock, rock        78.0   

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.482   0.721    0    -6.839     1       0.0321      0.731000   
1         0.502   0.912    1    -4.556     1       0.0564      0.000025   
2         0.364   0.457    4   -14.162     0       0.0675      0.290000   
3         0.398   0.413    1   -10.934     1       0.0388      0.082200   
4         0.534   0.803   10    -9.129     1       0.0743      0.741000   
5         0.805   0.558    0    -5.226     1       0.0410      0.164000   
6         0.511   0.769    4    -7.032     1       0.0420      0.181000   
7         0.283   0.716    1    -9.632     0       0.0986      0.447000   
8         0.711   0.362    3   -10.851     0       0.0381      0.272000   
9         0.723   0.863    2    -7.890     1       0.0338      0.038300   

   instrumentalness  liveness  valence    tempo            type  duration_ms  \
0          0.000000    0.1890    0.557   95.263  audio_features       369600   
1          0.000173    0.1060    0.720  116.761  audio_features       301920   
2          0.000106    0.9220    0.175  163.219  audio_features       337413   
3          0.000025    0.0891    0.331  133.574  audio_features       219147   
4          0.000061    0.3070    0.969  167.983  audio_features       161560   
5          0.000022    0.0546    0.965  114.950  audio_features       147600   
6          0.000000    0.0921    0.818  129.657  audio_features       160907   
7          0.000000    0.3990    0.828  201.960  audio_features       233000   
8          0.000000    0.1270    0.616  117.287  audio_features       194293   
9          0.031700    0.1280    0.931  136.302  audio_features       222813   

   time_signature  speechiness_binned  danceability_binned  \
0               4                   0                    1   
1               4                   0                    1   
2               4                   0                    1   
3               4                   0                    1   
4               4                   0                    1   
5               4                   0                    2   
6               4                   0                    1   
7               4                   0                    0   
8               4                   0                    2   
9               4                   0                    2   

   instrumentalness_binned                                    track_name  \
0                        0                          Like a Rolling Stone   
1                        0                       Smells Like Teen Spirit   
2                        0           A Day In The Life - Remastered 2009   
3                        0

In [19]:
def getSubjectivity(text):
  '''
  Getting the Subjectivity using TextBlob
  '''
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  '''
  Getting the Polarity using TextBlob
  '''
  return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
  '''
  Categorizing the Polarity & Subjectivity score
  '''
  if task == "subjectivity":
    if score < 1/3:
      return "low"
    elif score > 1/3:
      return "high"
    else:
      return "medium"
  else:
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

def sentiment_analysis(df, text_col):
  '''
  Perform sentiment analysis on text
  ---
  Input:
  df (pandas dataframe): Dataframe of interest
  text_col (str): column of interest
  '''
  df['subjectivity'] = df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
  df['polarity'] = df[text_col].apply(getPolarity).apply(getAnalysis)
  return df

In [20]:
sentiment = sentiment_analysis(final_data, "track_name")
sentiment.head()

artist_names                      id  \
0       Bob Dylan  3AhXZa8sUQht0UEdBJgpGc   
1         Nirvana  5ghIJDpPoe3CfHMGu71E6T   
2     The Beatles  0hKRSZhUGEhKU6aNSPBACZ   
3  The Beach Boys  5Qt4Cc66g24QWwGP3YYV9y   
4     Chuck Berry  2QfiRTz5Yc8DdShCxG1tB2   

                                               genre  popularity  \
0  album rock, classic rock, country rock, folk, ...        71.0   
1     alternative rock, grunge, permanent wave, rock        81.0   
2  beatlesque, british invasion, classic rock, me...        83.0   
3  baroque pop, brill building pop, classic rock,...        72.0   
4      classic rock, rock, rock-and-roll, rockabilly        65.0   

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.482   0.721    0    -6.839     1       0.0321      0.731000   
1         0.502   0.912    1    -4.556     1       0.0564      0.000025   
2         0.364   0.457    4   -14.162     0       0.0675      0.290000   
3         0.398   0.413    1   -10.934     1       0.0388      0.082200   
4         0.534   0.803   10    -9.129     1       0.0743      0.741000   

   instrumentalness  liveness  valence    tempo            type  duration_ms  \
0          0.000000    0.1890    0.557   95.263  audio_features       369600   
1          0.000173    0.1060    0.720  116.761  audio_features       301920   
2          0.000106    0.9220    0.175  163.219  audio_features       337413   
3          0.000025    0.0891    0.331  133.574  audio_features       219147   
4          0.000061    0.3070    0.969  167.983  audio_features       161560   

   time_signature  speechiness_binned  danceability_binned  \
0               4                   0                    1   
1               4                   0                    1   
2               4                   0                    1   
3               4                   0                    1   
4               4                   0                    1   

   instrumentalness_binned                           track_name  \
0                        0                 Like a Rolling Stone   
1                        0              Smells Like Teen Spirit   
2                        0  A Day In The Life - Remastered 2009   
3                        0               Good Vibrations (Mono)   
4                        0                      Johnny B. Goode   

   track_popularity album_released_date                album_id  \
0                69          1965-08-30  6YabPKtZAjxwyWbuO9p4ZD   
1                78          1991-09-26  2guirTSEqLizK7j9i1MTTZ   
2                62          1967-06-01  6QaVfG1pHYl1z15ZxkvVDW   
3                20          1967-09-18  3yGRRfowVUIj29zEwgLYY7   
4                75          1959-07-01  6eedtCtCjibu80yOhylSGL   

                                          album_name album_type  \
0                               Highway 61 Revisited      album   
1                             Nevermind (Remastered)      album   
2  Sgt. Pepper's Lonely Hearts Club Band (Remaste...      album   
3                       Smiley Smile (Mono & Stereo)      album   
4                                    Berry Is On Top      album   

                    artist_id  \
0  ['74ASZWbe4lXaubB36ztrGX']   
1  ['6olE6TJLqED3rqDCT0FyPh']   
2  ['3WrFJ7ztbogyGnTHbHJFl2']   
3  ['3oDbviiivRWhXwIE8hxkVV']   
4  ['293zczrfYafIItmnmM3coR']   

                                         genres_list subjectivity  polarity  
0  [album, rock,, classic, rock,, country, rock,,...          low   Neutral  
1  [alternative, rock,, grunge,, permanent, wave,...          low   Neutral  
2  [beatlesque,, british, invasion,, classic, roc...          low   Neutral  
3  [baroque, pop,, brill, building, pop,, classic...         high  Positive  
4  [classic, rock,, rock,, rock-and-roll,, rockab...          low   Neutral

In [21]:
final_data.polarity.value_counts()

Neutral     7229
Positive    1577
Negative     926
Name: polarity, dtype: int64

In [22]:
def ohe_prep(df, column, new_name): 
    ''' 
    Create One Hot Encoded features of a specific column
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    column (str): Column to be processed
    new_name (str): new column name to be used
        
    Output: 
    tf_df: One-hot encoded features 
    '''
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [23]:
# One-hot encoding for the subjectivity 
subject_ohe = ohe_prep(sentiment, 'subjectivity','subject')
subject_ohe.iloc[0]

subject|high      0
subject|low       1
subject|medium    0
Name: 0, dtype: uint8

In [24]:
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(final_data['genres_list'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
# genre_df.drop(columns='genre|unknown')
genre_df.reset_index(drop = True, inplace=True)
genre_df.iloc[0]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


genre|abstract          0.000000
genre|accordion         0.000000
genre|acid              0.000000
genre|acoustic          0.000000
genre|action            0.000000
genre|adult             0.000000
genre|advocacy          0.000000
genre|african           0.000000
genre|afro              0.000000
genre|afrobeat          0.000000
genre|afrofuturism      0.000000
genre|afroperuana       0.000000
genre|afropop           0.000000
genre|afroswing         0.000000
genre|age               0.000000
genre|ahead             0.000000
genre|ai                0.000000
genre|alabama           0.000000
genre|alaska            0.000000
genre|album             0.243806
genre|albuquerque       0.000000
genre|alley             0.000000
genre|alt               0.000000
genre|alternatif        0.000000
genre|alternative       0.000000
genre|ambient           0.000000
genre|america           0.000000
genre|american          0.000000
genre|americana         0.000000
genre|anarcho           0.000000
genre|and 

In [27]:
# artist_pop distribution descriptive stats
print(final_data['popularity'].describe())

count    9732.000000
mean       56.426600
std        17.440880
min         0.000000
25%        46.000000
50%        58.000000
75%        68.083333
max       100.000000
Name: popularity, dtype: float64


In [29]:
# Normalization
pop = final_data[['popularity']].reset_index(drop = True)
scaler = MinMaxScaler()
pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns)
pop_scaled.head()

popularity
0        0.71
1        0.81
2        0.83
3        0.72
4        0.65

In [34]:
def create_feature_set(df, float_cols):
    '''
    Process spotify df to create a final set of features that will be used to generate recommendations
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    float_cols (list(str)): List of float columns that will be scaled
            
    Output: 
    final (pandas dataframe): Final set of features 
    '''
    
    # Tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['genres_list'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    #genre_df.drop(columns='genre|unknown') # drop unknown genre
    genre_df.reset_index(drop = True, inplace=True)
    
    # Sentiment analysis
    df = sentiment_analysis(df, "track_name")

    # One-hot Encoding
    subject_ohe = ohe_prep(df, 'subjectivity','subject') * 0.3
    polar_ohe = ohe_prep(df, 'polarity','polar') * 0.5
    key_ohe = ohe_prep(df, 'key','key') * 0.5
    mode_ohe = ohe_prep(df, 'mode','mode') * 0.5

    # Normalization
    # Scale popularity columns
    pop = df[["popularity","track_popularity"]].reset_index(drop = True)
    scaler = MinMaxScaler()
    pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns) * 0.2 

    # Scale audio columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    # Concanenate all features
    final = pd.concat([genre_df, floats_scaled, pop_scaled, subject_ohe, polar_ohe, key_ohe, mode_ohe], axis = 1)
    
    # Add song id
    final['id']=df['id'].values
    
    return final

In [35]:
# Save the data and generate the features
float_cols = final_data.dtypes[final_data.dtypes == 'float64'].index.values
final_data.to_csv("../data/allsong_data.csv", index = False)

# Generate features
complete_feature_set = create_feature_set(final_data, float_cols=float_cols)
complete_feature_set.to_csv("../data/complete_feature.csv", index = False)
complete_feature_set.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


genre|abstract  genre|accordion  genre|acid  genre|acoustic  genre|action  \
0             0.0              0.0         0.0             0.0           0.0   
1             0.0              0.0         0.0             0.0           0.0   
2             0.0              0.0         0.0             0.0           0.0   
3             0.0              0.0         0.0             0.0           0.0   
4             0.0              0.0         0.0             0.0           0.0   

   genre|adult  genre|advocacy  genre|african  genre|afro  genre|afrobeat  \
0          0.0             0.0            0.0         0.0             0.0   
1          0.0             0.0            0.0         0.0             0.0   
2          0.0             0.0            0.0         0.0             0.0   
3          0.0             0.0            0.0         0.0             0.0   
4          0.0             0.0            0.0         0.0             0.0   

   genre|afrofuturism  genre|afroperuana  genre|afropop  genre|afroswing  \
0                 0.0                0.0            0.0              0.0   
1                 0.0                0.0            0.0              0.0   
2                 0.0                0.0            0.0              0.0   
3                 0.0                0.0            0.0              0.0   
4                 0.0                0.0            0.0              0.0   

   genre|age  genre|ahead  genre|ai  genre|alabama  genre|alaska  genre|album  \
0        0.0          0.0       0.0            0.0           0.0     0.243806   
1        0.0          0.0       0.0            0.0           0.0     0.000000   
2        0.0          0.0       0.0            0.0           0.0     0.000000   
3        0.0          0.0       0.0            0.0           0.0     0.000000   
4        0.0          0.0       0.0            0.0           0.0     0.000000   

   genre|albuquerque  genre|alley  genre|alt  genre|alternatif  \
0                0.0          0.0        0.0               0.0   
1                0.0          0.0        0.0               0.0   
2                0.0          0.0        0.0               0.0   
3                0.0          0.0        0.0               0.0   
4                0.0          0.0        0.0               0.0   

   genre|alternative  genre|ambient  genre|america  genre|american  \
0           0.000000            0.0            0.0             0.0   
1           0.334953            0.0            0.0             0.0   
2           0.000000            0.0            0.0             0.0   
3           0.000000            0.0            0.0             0.0   
4           0.000000            0.0            0.0             0.0   

   genre|americana  genre|anarcho  genre|and  genre|andaluz  genre|andina  \
0              0.0            0.0   0.000000            0.0           0.0   
1              0.0            0.0   0.000000            0.0           0.0   
2              0.0            0.0   0.000000            0.0           0.0   
3              0.0            0.0   0.000000            0.0           0.0   
4              0.0            0.0   0.416578            0.0           0.0   

   genre|anglia  genre|anime  genre|anti  genre|antideutsche  \
0           0.0          0.0         0.0                 0.0   
1           0.0          0.0         0.0                 0.0   
2           0.0          0.0         0.0                 0.0   
3           0.0          0.0         0.0                 0.0   
4           0.0          0.0         0.0                 0.0   

   genre|appalachian  genre|arab  genre|aragonesa  genre|area  genre|arena  \
0                0.0         0.0              0.0         0.0          0.0   
1                0.0         0.0              0.0         0.0          0.0   
2                0.0         0.0              0.0         0.0          0.0   
3                0.0         0.0              0.0         0.0          0.0   
4                0.0         0.0              0.0         0

In [36]:
# Drop song duplicates
def drop_duplicates(df):
    '''
    Drop duplicate songs
    '''
    df['artists_song'] = df.apply(lambda row: row['artist_name']+row['track_name'],axis = 1)
    return df.drop_duplicates('artists_song')



In [37]:
# Select useful columns
def select_cols(df):
       '''
       Select useful columns
       '''
       return df[['artist_name','id','danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', "artist_pop", "genres", "track_pop"]]


In [38]:
def genre_preprocess(df):
    '''
    Preprocess the genre data
    '''
    df['genres_list'] = df['genres'].apply(lambda x: x.split(" "))
    return df


In [39]:
def playlist_preprocess(df):
    '''
    Preprocess imported playlist
    '''
    df = drop_duplicates(df)
    df = select_cols(df)
    df = genre_preprocess(df)

    return df

In [40]:
### This is the test data
playlistDF_test = pd.read_csv("../data/test_playlist.csv")
playlistDF_test = playlist_preprocess(playlistDF_test)
playlistDF_test.head()

artist_name                      id  danceability  energy  key  \
0       The Killers  7oK9VyNzrYvRFo7nQEYkWN         0.356   0.924    1   
1           Rihanna  6qn9YLKt13AGvpq9jfO8py         0.734   0.766    1   
2  American Authors  5j9iuo3tMmQIfnEEQOOjxh         0.670   0.905    2   
3      Clean Bandit  5HuqzFfq2ulY1iBAW5CxLe         0.799   0.586   11   
4               Sia  4VrWlk8IQxevMvERoX08iC         0.399   0.787    1   

   loudness  mode  speechiness  acousticness  instrumentalness  liveness  \
0    -3.740     1       0.0808       0.00101          0.000000    0.0953   
1    -4.485     1       0.0383       0.02500          0.001380    0.1080   
2    -2.385     1       0.0339       0.06250          0.000151    0.0577   
3    -6.735     1       0.0377       0.16200          0.000002    0.1930   
4    -2.880     1       0.0499       0.01970          0.000061    0.0685   

   valence    tempo  artist_pop  \
0    0.232  148.017          80   
1    0.600  127.986          90   
2    0.516  100.021          70   
3    0.549  120.970          80   
4    0.572  117.089          89   

                                              genres  track_pop  \
0  alternative_rock dance_rock modern_rock perman...         78   
1  barbadian_pop dance_pop pop pop_rap urban_cont...         77   
2  indie_poptimism modern_alternative_rock modern...          0   
3  dance_pop edm pop pop_dance tropical_house uk_...         53   
4                australian_dance australian_pop pop         81   

                                         genres_list  
0  [alternative_rock, dance_rock, modern_rock, pe...  
1  [barbadian_pop, dance_pop, pop, pop_rap, urban...  
2  [indie_poptimism, modern_alternative_rock, mod...  
3  [dance_pop, edm, pop, pop_dance, tropical_hous...  
4            [australian_dance, australian_pop, pop]

In [41]:
def generate_playlist_feature(complete_feature_set, playlist_df):
    '''
    Summarize a user's playlist into a single vector
    ---
    Input: 
    complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
    playlist_df (pandas dataframe): playlist dataframe
        
    Output: 
    complete_feature_set_playlist_final (pandas series): single vector feature that summarizes the playlist
    complete_feature_set_nonplaylist (pandas dataframe): 
    '''
    
    # Find song features in the playlist
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]
    # Find all non-playlist song features
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]
    complete_feature_set_playlist_final = complete_feature_set_playlist.drop(columns = "id")
    return complete_feature_set_playlist_final.sum(axis = 0), complete_feature_set_nonplaylist

In [42]:
# Generate the features
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set, playlistDF_test)

In [43]:
# Non-playlist features
complete_feature_set_nonplaylist.head()

genre|abstract  genre|accordion  genre|acid  genre|acoustic  genre|action  \
0             0.0              0.0         0.0             0.0           0.0   
1             0.0              0.0         0.0             0.0           0.0   
2             0.0              0.0         0.0             0.0           0.0   
3             0.0              0.0         0.0             0.0           0.0   
4             0.0              0.0         0.0             0.0           0.0   

   genre|adult  genre|advocacy  genre|african  genre|afro  genre|afrobeat  \
0          0.0             0.0            0.0         0.0             0.0   
1          0.0             0.0            0.0         0.0             0.0   
2          0.0             0.0            0.0         0.0             0.0   
3          0.0             0.0            0.0         0.0             0.0   
4          0.0             0.0            0.0         0.0             0.0   

   genre|afrofuturism  genre|afroperuana  genre|afropop  genre|afroswing  \
0                 0.0                0.0            0.0              0.0   
1                 0.0                0.0            0.0              0.0   
2                 0.0                0.0            0.0              0.0   
3                 0.0                0.0            0.0              0.0   
4                 0.0                0.0            0.0              0.0   

   genre|age  genre|ahead  genre|ai  genre|alabama  genre|alaska  genre|album  \
0        0.0          0.0       0.0            0.0           0.0     0.243806   
1        0.0          0.0       0.0            0.0           0.0     0.000000   
2        0.0          0.0       0.0            0.0           0.0     0.000000   
3        0.0          0.0       0.0            0.0           0.0     0.000000   
4        0.0          0.0       0.0            0.0           0.0     0.000000   

   genre|albuquerque  genre|alley  genre|alt  genre|alternatif  \
0                0.0          0.0        0.0               0.0   
1                0.0          0.0        0.0               0.0   
2                0.0          0.0        0.0               0.0   
3                0.0          0.0        0.0               0.0   
4                0.0          0.0        0.0               0.0   

   genre|alternative  genre|ambient  genre|america  genre|american  \
0           0.000000            0.0            0.0             0.0   
1           0.334953            0.0            0.0             0.0   
2           0.000000            0.0            0.0             0.0   
3           0.000000            0.0            0.0             0.0   
4           0.000000            0.0            0.0             0.0   

   genre|americana  genre|anarcho  genre|and  genre|andaluz  genre|andina  \
0              0.0            0.0   0.000000            0.0           0.0   
1              0.0            0.0   0.000000            0.0           0.0   
2              0.0            0.0   0.000000            0.0           0.0   
3              0.0            0.0   0.000000            0.0           0.0   
4              0.0            0.0   0.416578            0.0           0.0   

   genre|anglia  genre|anime  genre|anti  genre|antideutsche  \
0           0.0          0.0         0.0                 0.0   
1           0.0          0.0         0.0                 0.0   
2           0.0          0.0         0.0                 0.0   
3           0.0          0.0         0.0                 0.0   
4           0.0          0.0         0.0                 0.0   

   genre|appalachian  genre|arab  genre|aragonesa  genre|area  genre|arena  \
0                0.0         0.0              0.0         0.0          0.0   
1                0.0         0.0              0.0         0.0          0.0   
2                0.0         0.0              0.0         0.0          0.0   
3                0.0         0.0              0.0         0.0          0.0   
4                0.0         0.0              0.0         0

In [44]:
# Summarized playlist features
complete_feature_set_playlist_vector

genre|abstract          0.000000
genre|accordion         0.000000
genre|acid              0.000000
genre|acoustic          0.000000
genre|action            0.000000
genre|adult             0.000000
genre|advocacy          0.000000
genre|african           0.000000
genre|afro              0.000000
genre|afrobeat          0.000000
genre|afrofuturism      0.000000
genre|afroperuana       0.000000
genre|afropop           0.000000
genre|afroswing         0.000000
genre|age               0.000000
genre|ahead             0.000000
genre|ai                0.000000
genre|alabama           0.000000
genre|alaska            0.000000
genre|album             0.000000
genre|albuquerque       0.000000
genre|alley             0.000000
genre|alt               0.000000
genre|alternatif        0.000000
genre|alternative       0.371854
genre|ambient           0.000000
genre|america           0.000000
genre|american          0.000000
genre|americana         0.000000
genre|anarcho           0.000000
genre|and 

In [45]:
def generate_playlist_recos(df, features, nonplaylist_features):
    '''
    Generated recommendation based on songs in aspecific playlist.
    ---
    Input: 
    df (pandas dataframe): spotify dataframe
    features (pandas series): summarized playlist feature (single vector)
    nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Output: 
    non_playlist_df_top_40: Top 40 recommendations for that playlist
    '''
    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    # Find cosine similarity between the playlist and the complete song set
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    
    return non_playlist_df_top_40

In [46]:
# Genreate top 10 recommendation
recommend = generate_playlist_recos(final_data, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)
recommend.head(10)

/var/folders/2w/1gmdxhl509q5y8phddv3y6sc0000gn/T/ipykernel_3769/1711594805.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]


artist_names                      id  \
1790       Gwen Stefani  5R8dQOPq8haW94K7mgERlO   
1185            Madonna  7aTCCNpHJD2Jgv0LLVJb8Y   
8802      Ariana Grande  7F8eSGduNKaqn16evCtdB0   
7088               P!nk  4mdyVTV7Tr5YDFnD2kvSM4   
4110         Katy Perry  3dUMuxNadGxCE3qXuhCwqa   
2043            Madonna  2gQaQUhDCNGfBVXTvxAmXQ   
1424  Justin Timberlake  2QOMGq8wVTZbLmh7McrvgF   
761             Madonna  4IRHwIZHzlHT1FQpRa5RdE   
1714        Miley Cyrus  2vwlzO0Qp8kfEtzTsCXfyE   
1322           No Doubt  7eXFZCiP9GqdMUashk96UF   

                                                  genre  popularity  \
1790                                     dance pop, pop        73.0   
1185                                     dance pop, pop        77.0   
8802                                     dance pop, pop        89.0   
7088                                     dance pop, pop        79.0   
4110                                     dance pop, pop        84.0   
2043                                     dance pop, pop        77.0   
1424                                     dance pop, pop        80.0   
761                                      dance pop, pop        77.0   
1714                                     dance pop, pop        82.0   
1322  alternative rock, dance pop, dance rock, perma...        67.0   

      danceability  energy  key  loudness  mode  speechiness  acousticness  \
1790         0.851   0.806    4    -4.620     1       0.0787      0.118000   
1185         0.719   0.670    9    -9.113     1       0.0903      0.243000   
8802         0.498   0.913    9    -6.167     1       0.0323      0.006350   
7088         0.579   0.554    0    -7.080     1       0.0587      0.017500   
4110         0.578   0.777    0    -7.572     1       0.2430      0.261000   
2043         0.511   0.942    0   -10.407     1       0.0429      0.174000   
1424         0.437   0.717    0    -4.803     1       0.0390      0.077900   
761          0.559   0.473    5    -7.712     1       0.0279      0.446000   
1714         0.530   0.422    5    -6.262     1       0.0342      0.407000   
1322         0.639   0.632    9    -7.589     1       0.0290      0.000904   

      instrumentalness  liveness  valence    tempo            type  \
1790          0.000002    0.1210    0.787  118.999  audio_features   
1185          0.022300    0.0618    0.960  144.041  audio_features   
8802          0.037100    0.0959    0.701  136.965  audio_features   
7088          0.022100    0.1010    0.838  185.183  audio_features   
4110          0.003180    0.2230    0.650  205.749  audio_features   
2043          0.000019    0.1390    0.498   97.909  audio_features   
1424          0.002740    0.1080    0.547  178.085  audio_features   
761           0.001410    0.1540    0.397  121.227  audio_features   
1714          0.000000    0.1070    0.349  119.964  audio_features   
1322          0.041100    0.1150    0.350  114.993  audio_features   

      duration_ms  time_signature  speechiness_binned  danceability_binned  \
1790       237200               4                   0                    2   
1185       270293               4                   0                    2   
8802       170907               4                   0                    1   
7088       241467               4                   0                    1   
4110       205200               4                   0                    1   
2043       393667               4                   0                    1   
1424       215907               4                   0                    1   
761        192827               4                   0                    1   
1714       221360               4                   0                    1   
1322       306532               4                   0                    1   

      instrumentalness_binned                                   track_name  \
1790                        0                                   Poker Face   
1185                  